In [1]:
import sys
import os
import numpy as np


notebook_dir = os.path.dirname(os.path.abspath("__file__"))
src_path = os.path.join(notebook_dir, "..", "code")
if src_path not in sys.path:
    sys.path.append(src_path)

from PCA import PCA

## Generate and visualize random 2D Gaussian samples

In [2]:
import numpy as np
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA as coucoujesuisunePCA

#fix seed
np.random.seed(650)

mean = [10,6]
covariance = [[10,7], [7,10]]
num_points = 30

X = np.random.multivariate_normal(mean, covariance, num_points)


pca = PCA(nb_components=1)

pca.fit(X)
projections = pca.transform(X)
eigenvector = pca.components

In [3]:
df = pd.DataFrame(X, columns=["x", "y"])

fig = px.scatter(
    df,
    x="x",
    y="y",
    title="Visualisation des points d'une distribution gaussienne 2D",
    labels={"x": "Dimension X", "y": "Dimension Y"},
    template="plotly_white"
)

fig.update_traces(marker=dict(size=8, color='blue', line=dict(width=1, color='black')))
fig.update_layout(
    title_font_size=20,
    xaxis_title="Dimension X",
    yaxis_title="Dimension Y")

fig.show()


In [4]:
from utils_visualization import plot_fig_projections

mean_X = np.mean(X, axis=0)
projections_2d = projections @ eigenvector.T + mean_X

plot_fig_projections(X, projections_2d)

## Obtain same plot with PPCA with given sigma, then classical PPCA

### Given sigma2:

In [5]:
from PCA import PPCA
from utils_stats import get_likelihood

sigma2 = 10
ppca_sigma_fixed = PPCA(nb_components=1, sigma2 = sigma2)

ppca_sigma_fixed.fit(X)
projections_sigma_fixed = ppca_sigma_fixed.transform(X)
W = ppca_sigma_fixed.W
eigenvector = ppca_sigma_fixed.components

projections_ppca_sigma_fixed_2d = projections_sigma_fixed @ W.T + mean_X
plot_fig_projections(X, projections_ppca_sigma_fixed_2d, line_scale = 4.5)
print(get_likelihood(X, W, sigma2=sigma2))


-148.3277526381232


### With the sigma square that maximizes the likelihood of the model

In [6]:
from PCA import PPCA

ppca = PPCA(nb_components=1)

ppca.fit(X)
projections = ppca.transform(X)
W = ppca.W
eigenvector = ppca.components

projections_ppca_2d = projections @ W.T + mean_X
plot_fig_projections(X, projections_ppca_2d, line_scale = 1.9)
print(get_likelihood(X, W, sigma2=ppca.sigma2))
print(ppca.sigma2)

-138.0472207680325
2.3432574887761


### Test with slightly different distribution

In [14]:
mean = [10,6]
covariance = [[10,0], [0,10]]
num_points = 30

np.random.seed(650)
X = np.random.multivariate_normal(mean, covariance, num_points)
ppca = PPCA(nb_components=1)

ppca.fit(X)
projections = ppca.transform(X)
W = ppca.W
eigenvector = ppca.components

projections_ppca_2d = projections @ W.T + mean_X
plot_fig_projections(X, projections_ppca_2d, line_scale = 3)
print(get_likelihood(X, W, sigma2=ppca.sigma2))
print(ppca.sigma2)


-148.147389066989
7.788223336216862
